<a href="https://colab.research.google.com/github/changhorang/My-Project-1/blob/master/Attention_is_All_You_Need_Tutorial_(German_English).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Attention is All You Need Tutorial (NIPS 2017) 실습**
- Transformer 논문의 내용 기반

BLEU Score 계산을 위한 라이브러리 업데이트
- [Restart Runtime] 버튼을 눌러 런타임을 재시

In [1]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 1.2MB 20.5MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


**데이터 전처리 (Preprocessing)**
- spacy 라이브러리 : 문장의 토큰화(tokenization), 태깅 (tagging) 드으이 전처리 기능을 위한 라이브러리
 - 영어(English)와 독일(Deutsch) 전처리 모듈 설

In [2]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [3]:
import spacy

spacy_en = spacy.load('en') # 영어 토큰화
spacy_de = spacy.load('de') # 독일어 토큰화

In [4]:
# 간단히 토큰화 기능 사용
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


- 영어 및 독일어 토큰화 함수 정의

In [5]:
# 독일어 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

- 필드(field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시
- Seq2Seq 모델과는 다르게 batch_first 속성의 값을 True로 설정
- 번역 목표
 - 소스(SRC) : 독일어
 - 목표(TRF) : 영어

In [9]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

- 대표적인 영어-독어 번역 데이터셋인 Multi30k 불러오기

In [13]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

In [14]:
print(f"학습 데이터셋(training dataset) 크기 : {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기 : {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(test dataset) 크기 : {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기 : 29000개
평가 데이터셋(validation dataset) 크기 : 1014개
테스트 데이터셋(test dataset) 크기 : 1000개


In [15]:
# 학습 데이터중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


- 필드(field) 객체의 build_vocab 메서드를 이용해 영어와 독어의 단어 사전을 생성
 - 최소 2번 이상 등장한 단어만을 선택

In [16]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


In [17]:
print(TRG.vocab.stoi["abcabc"]) # 없는단어 : 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding) : 1
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


- 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 함
 - 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋음
 - 이를 위해 Bucketlterator 사용
 - 배치 크기(batch size) : 128

In [20]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

# 일반적인 데이터 로더 (data loader)의 iterator와 유사하게 사용가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE, device = device
)

In [24]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f"첫 번째 배치 크기:{src.shape}")

  # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
  for i in range(src.shape[1]):
    print(f"index {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

  # 첫 번째 배치만 확인
  break

첫 번째 배치 크기:torch.Size([128, 33])
index 0: 2
index 1: 5
index 2: 32
index 3: 472
index 4: 5
index 5: 312
index 6: 20
index 7: 123
index 8: 4
index 9: 3
index 10: 1
index 11: 1
index 12: 1
index 13: 1
index 14: 1
index 15: 1
index 16: 1
index 17: 1
index 18: 1
index 19: 1
index 20: 1
index 21: 1
index 22: 1
index 23: 1
index 24: 1
index 25: 1
index 26: 1
index 27: 1
index 28: 1
index 29: 1
index 30: 1
index 31: 1
index 32: 1


### Multi Head Attention 아키텍쳐
- 어텐션(attention)은 세 가지 요소를 입력으로 받음
 - 쿼리(queries)
 - 키(keys)
 - 값(values)
 - 현재 구현에서는 Query, Key, Value의 차원이 모두 동일

- 하이퍼 파라미터 (hyperparameter)
 - hidden_dim : 하나의 단어에 대한 임베딩 차원
 - n_heads : head의 개수 = scaled dot-product attention의 개수
 - dropout_ratio : 드롭아웃 비율

In [25]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim # 임베딩 차원
    self.n_heads = n_heads # head의 개수 : 서로 다른 어텐션(attention) 컨셉의 수
    self.head_dim = hidden_dim // n_heads # 각 head에서의 임베딩 차원

    self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
    self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
    self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloarTensor([self.head_dim])).to(device)

def forward(self, query, key, value, mask = None):

  batch_size = query.shape[0]

  # query : [batch_size, query_len, hidden_dim]
  # key : [batch_size, key_len, hidden_dim]
  # value : [batch_size, value_len, hidden_dim]

  Q = self.fc_q(query)
  K = self.fc_k(key)
  V = self.fc_v(value)

  # hidden_dim -> n_heads X head_dim 형태로 변형
  # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
  Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
  K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
  V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

  # Q: [batch_size, n_heads, query_len, head_dim]
  # K: [batch_size, n_heads, key_len, head_dim]
  # V: [batch_size, n_heads, value_len, head_dim]
  
  # Attention Energy 계산
  energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale

  # energy : [batch_size, n_heads, query_len, key_len]

  # mask를 사용하는 경우
  if mask is not None:
    # mask 값이 0인 부분을 -1e10으로 채우기
    energy = energy.masked_fill(mask==0, -1e10)

  # attention 스코어 계산: 각 단어에 대한 확률 값
  attention = torch.softmax(energy, dim=-1)

  # attention: [batch_size, n_heads, query_len, key_len]

  # 여기에서 Scaled Dot-Product Attention을 계산
  x = torch.matmul(self.dropout(attention), V)

  # x: [batch_size, n_heads, query_len, head_dim]
  
  x = x.permute(0,2,1,3).contiguous()

  # x: [batch_size, query_len, n_heads, head_dim]

  x = x.view(batch_size, -1, self.hidden_dim)

  # x: [batch_size, query_len, hidden_dim]

  x = self.fc_o(x)

  # x: [batch_size, query_len, hidden_dim]

  return x, attention

### Position-wise Feedforward 아키텍쳐
- 입력과 출력의 차원이 동일
- 하이퍼 파라미터
 - hidden_dim : 하나의 단어에 대한 임베딩 차원
 - pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
 - dropout_ratio: dropout 비율

In [26]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hidden_dim, pf_dim, dropout_ratio):
    super().__init__()

    self.fc_1 = nn.Linear(hidden_dim, pf_dim)
    self.fc_2 = nn.Linear(pf_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, x):

    # x: [batch_size, seq_len, hidden_dim]

    x = self.dropout(torch.relu(self.fc_1(x)))

    # x: [batch_size, seq_len, pf_dim]

    x = self.fc_2(x)

    # x: [batch_size, seq_len, hidden_dim]

    return x

# 인코더(Encoder) 레이어 아키텍처

- 하나의 인코더 레이어에 대해 정의
 - 입력과 출력의 차원이 동일
 - 이러한 특징을 이용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해 사용

- 하이퍼 파라미터
 - hidden_dim: 하나의 단어에 대한 임베딩 차원
 - n_heads: head의 개수 = scaled dot-product attention의 개수
 - pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
 - dropout_ratio: dropout 비율
- <pad> 토큰에 대해 mask 값을 0으로 설정

In [28]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
  def forward(self, src, src_mask):

    # src: [batch_size, src_len, hidden_dim]
    # src_mask: [batch_size, src_len]

    # self attention
    # 필요한 경우 mask 행렬을 이용해 attention할 단어를 조절 가능
    _src, _ = self.self_attention(src, src, src, src_mask)

    # dropout, residual connection and layer norm
    src = self.self_attn_layer_norm(src + self.dropout(_src))

    # src : [batch_size, src_len, hidden_dim]

    # position-wise feedforward
    _src = self.positionwise_feedforward(src)

    # dropout, residual and layer norm
    src = self.ff_layer_norm(src + self.dropout(_src))

    # src: [batch_size, src_len, hidden_dim]

    return src

### Encoder 아키텍처
- 전체 인코더 아키텍처를 정의
- 하이퍼 파라미터
 - input_dim: 하나의 단어에 대한 원핫인코딩 차원
 - hidden_dim: 하나의 단어에 대한 임베딩 차원
 - n_layers: 내부적으로 사용할 인코더 레이어의 개수
 - n_heads: head의 개수 = scaled dot-product attention 개수
 - pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
 - dropout_ratio: dropout 비율
 - max_length: 문장 내 최대 단어 개수

- 원본 논문과는 다르게 위치 임베딩을 학습하는 형태로 구현
 - BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식

- <pad> 토큰에 대해 mask 값을 0으로 설정

In [30]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, src, src_mask):

    # src: [batch_size, src_len]
    # src_mask: [batch_size, src_len]

    batch_size = src.shape[0]
    src_len = src.shape[1]

    pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # pos: [batch_size, src_len]

    # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
    src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

    # src: [batch_size, src_len, hidden_dim]

    # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
    for layer in self.layers:
      src = layer(src, src_mask)

    # src: [batch_size, src_len, hidden_dim]

    return src # 마지막 레이어의 출력을 반환